In [1]:
import numpy as np
import healpy as hp
import mm_constrained_realizations as mmcr
import matplotlib.pyplot as plt

%matplotlib inline

In [16]:
import planck_colormap
cm = planck_colormap.colormap()
import time

In [59]:
### set up - params
nside = 64
lmax = 3*nside//2
#fwhm = (160/60)*np.pi/180
### importing maps
weights_map = hp.read_map('Commander-Ruler/input_files/full_weights_map_08_0064.fits')
cl = np.load('Commander-Ruler/input_files/cls_PlanckPR2_TT_lowp_lensing_lensed.npy')
tmap = hp.read_map('Commander-Ruler/input_files/commander_t_map.fits')
noise_cov = hp.read_map('Commander-Ruler/input_files/commander_noise_cov.fits')
mask = hp.read_map('Commander-Ruler/input_files/commander_mask.fits')
### IMPORTANT ####
# as seen in prepping input files notebook
# bl is 1 for every ell at this scale so we wont smooth
# bumping nosie_cov low end
# noise_cov += 0

NSIDE = 64
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 64
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 64
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 64
ORDERING = RING in fits file
INDXSCHM = IMPLICIT


In [60]:
### initializing CR
cr = mmcr.ConstrainedRealizations(nside,weights_map,mask=mask)

In [61]:
# setting covariances
cr.set_signal_cov(cl)
cr.set_noise_cov(noise_cov)

In [62]:
# calculating one delta for convergence study
delta = cr.gen_delta()

In [63]:
t0 = time.time()
lamb_0 = 1e6
tp = 1e-4
cr.set_cooling_schedule(lamb_0,tp)
flm = cr.gen_constrained_realization(delta_fix=delta)

dt = np.round(time.time() - t0,decimals=1)
print("lamb_ 0 = {}, tp = {}, dt = {}s".format(lamb_0, tp, dt))

lamb_ 0 = 1000000.0, tp = 0.0001, dt = 2.5s


In [76]:
# cooling schedule params to vary over
lamb_0_list = [1e10]
tp_list = [1e-6,1e-7]

In [ ]:
cl_list = []
for lamb_0 in lamb_0_list:
    for tp in tp_list:
        t0 = time.time()
        cr.set_cooling_schedule(lamb_0,tp)
        flm = cr.gen_constrained_realization(delta_fix=delta)
        dt = np.round(time.time() - t0,decimals=1)
        cl_cr = hp.alm2cl(flm,lmax=cr.params.lmax)
        cl_list.append([cl_cr,"{}; {}; {}s".format(np.log10(lamb_0),np.log10(tp),dt)])

In [ ]:
for item in cl_list:
    plt.plot(item[0],label=item[1])

plt.xscale('log')
plt.xlim(2,cr.params.lmax)
plt.legend(loc='upper right')
plt.show()

In [ ]:
for i in range(len(cl_list)-1):
    plt.plot(np.abs(cl_list[i][0] - cl_list[-1][0])/cl_list[-1][0],label=cl_list[i][1])

plt.xscale('log')
plt.yscale('log')
plt.xlim(2,cr.params.lmax)
plt.legend(loc='lower left')
plt.show()

### Conclustions
